In [ ]:
!pip install flask pyngrok


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p /content/templates

In [ ]:
# Writing index.html
with open('/content/templates/index.html', 'w') as f:
    f.write("""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI-Powered Q&A System</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <style>
        body {
            background: linear-gradient(135deg, #1f4037, #99f2c8);
            color: white;
            text-align: center;
        }
        .container {
            margin-top: 10%;
            padding: 30px;
            background: rgba(0, 0, 0, 0.7);
            border-radius: 10px;
            box-shadow: 0 0 15px rgba(255, 255, 255, 0.2);
        }
        .btn-primary {
            background-color: #4CAF50;
            border: none;
            font-size: 18px;
            padding: 10px 20px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1 class="mb-4">AI-Powered Q&A System</h1>
        <form action="/answer" method="post">
            <div class="mb-3">
                <label for="question" class="form-label">Enter Your Question:</label>
                <input type="text" class="form-control" id="question" name="question" required>
            </div>
            <button type="submit" class="btn btn-primary">Get Answer</button>
        </form>
    </div>
</body>
</html>
""")

# Writing result.html
with open('/content/templates/result.html', 'w') as f:
    f.write("""<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI Answer</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <style>
        body {
            background: linear-gradient(135deg, #2c3e50, #4ca1af);
            color: white;
            text-align: center;
        }
        .container {
            margin-top: 10%;
            padding: 30px;
            background: rgba(0, 0, 0, 0.8);
            border-radius: 10px;
            box-shadow: 0 0 15px rgba(255, 255, 255, 0.2);
        }
        .btn-primary {
            background-color: #3498db;
            border: none;
            font-size: 18px;
            padding: 10px 20px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1 class="mb-4">AI-Generated Answer</h1>
        <h4><strong>Question:</strong> {{ question }}</h4>
        <hr>
        <h4><strong>Answer:</strong></h4>
        <p class="lead">{{ answer }}</p>
        <a href="/" class="btn btn-primary mt-3">Ask Another Question</a>
    </div>
</body>
</html>
""")


In [ ]:
import os
import time
import threading
from flask import Flask, render_template, request
from transformers import pipeline
from pyngrok import ngrok

# ✅ Set ngrok authentication token
NGROK_AUTH_TOKEN = "2sfeci6TxZbWK2dzigOmSwpovJR_2hUp5dLV6hMXSxyMoe9pZ"  # Replace with your actual token
if not NGROK_AUTH_TOKEN:
    raise ValueError("Error: ngrok authentication token is missing!")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.kill()  # Disconnect existing tunnels

# ✅ Predefined Context (AI & Information Retrieval)
CONTEXT = """
Artificial Intelligence (AI) is a field of computer science focused on creating systems that can perform tasks typically requiring human intelligence.
In the context of information retrieval (IR), AI enhances search accuracy by understanding user intent and ranking results efficiently.

Information retrieval systems fetch relevant data from massive datasets. Traditional IR methods relied on keyword matching, while AI-driven
approaches use machine learning, vector embeddings, and transformer-based models like BERT and T5 to improve search quality.

Modern AI-powered IR systems leverage deep learning to provide context-aware responses, personalized recommendations, and efficient document ranking.
Applications of AI-driven IR include search engines, chatbots, virtual assistants, and enterprise knowledge management systems.

Key advancements in AI for IR:
- **Neural Search Models**: AI-based semantic search that understands word relationships.
- **Conversational AI**: Virtual assistants like ChatGPT provide real-time Q&A.
- **Recommendation Engines**: AI predicts user preferences to refine search results.
- **Multimodal Search**: Future AI search engines will integrate text, images, and video.

AI-driven information retrieval is transforming industries such as healthcare, finance, and cybersecurity by delivering intelligent, data-driven insights.
"""

# ✅ Load Hugging Face Models (Optimized for Stability)
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")  # Short answer extraction
text_gen = pipeline("text2text-generation", model="google/flan-t5-large")  # Lightweight long-form answer generation

# ✅ Flask App Setup
app = Flask(__name__)

@app.route('/')
def home():
    return render_template("index.html")

@app.route('/answer', methods=['POST'])
def get_answer():
    question = request.form['question']
    if not question:
        return "Error: Please provide a valid question!"

    try:
        # Extract short answer from QA model
        short_answer = qa_pipeline(question=question, context=CONTEXT)['answer']

        # Expand answer using T5 text generation model
        prompt = f"Provide a detailed explanation for: {question} Answer: {short_answer}"
        long_answer = text_gen(prompt, max_length=200)[0]['generated_text']
    except Exception as e:
        long_answer = f"Error processing request: {str(e)}"

    return render_template("result.html", question=question, answer=long_answer)  # Show detailed answer

# ✅ Open ngrok tunnel
public_url = ngrok.connect(5000).public_url
print(f"🌍 Public URL: {public_url}")

# ✅ Start Flask server in a separate thread
def run_flask():
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

t = threading.Thread(target=run_flask, daemon=True)
t.start()

time.sleep(2)  # Give Flask time to initialize
print("✅ Flask Server is running! Access it via:", public_url)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


🌍 Public URL: https://09c4-34-125-138-14.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


✅ Flask Server is running! Access it via: https://09c4-34-125-138-14.ngrok-free.app


In [ ]:
!pkill -f ngrok
!fuser -k 5000/tcp


In [ ]:
!fuser -k 5000/tcp
